In [36]:
#!pip install plotly
#!pip install pandas
#!pip install kaleido

In [37]:
import plotly.express as px
import pandas as pd
import plotly.io as pio
import requests
import datetime
from bs4 import BeautifulSoup

datetime_format = "" #沒填日期預設讀取昨日的資料 格式:yyyymmdd


time_del = datetime.timedelta(days=1) 
new_dt = datetime.date.today() - time_del
if(datetime_format == ""):
    datetime_format = new_dt.strftime("%Y%m%d")


url = "https://www.twse.com.tw/zh/exchangeReport/MI_INDEX"
par = {
    "response": "html",
    "date": datetime_format,
    "type": "ALLBUT0999"
}

HTML_respinse = requests.post(url, par)
HTML = BeautifulSoup(HTML_respinse.text)
#price_HTML = HTML.find("table")

for num in range(0,6):
    thead_list = []
    tbody_list = []
    price_HTML = HTML.find_all("table")[num]
    title = price_HTML.find("th").text
    thead_HTML = price_HTML.find("thead")
    tbody_HTML = price_HTML.find("tbody")

    tr = thead_HTML.find_all("tr")[1]
    for td in tr.find_all("td"):
        thead_list.append(td.text)

    tbody_all_row = tbody_HTML.find_all("tr")

    for i in tbody_all_row:
        row_list = []
        for td in i.find_all("td"):
            row_list.append(td.text)
        tbody_list.append(row_list)
    
    shareprice_df = pd.DataFrame(data = tbody_list,columns=thead_list)
    for i in shareprice_df:
        shareprice_df['特殊處理註記'] = title

    shareprice_df.to_excel(f"{num}.xlsx")

df0 = pd.read_excel('0.xlsx')
df1 = pd.read_excel('1.xlsx')
df2 = pd.read_excel('2.xlsx')
#df3 = pd.read_excel('3.xlsx')
#df4 = pd.read_excel('4.xlsx')
#df5 = pd.read_excel('5.xlsx')


df = pd.concat([df0,df1,df2], ignore_index=False)
df.to_excel('merge.xlsx')




df = pd.read_excel("./merge.xlsx")

#開始繪圖
fig = px.treemap(df, path=['指數'], values='漲跌百分比(%)',color_continuous_scale='Geyser',color='漲跌百分比(%)',height = 1080,width = 1080,)

fig.update_traces(textposition='middle center',
                          textfont_size=24,
                          textinfo='label+text+value+percent parent'
                          )
fig.data[0].texttemplate = "%{label}<br>%{value}%"
pio.write_image(fig, 'img.png')



#透過 LINE 發送圖片
token = '4HDRnoNBsqpfnud05u4VAus7HKOjXxJywer6D3HzeNB'

message = '下午茶配指數~'
headers = { "Authorization": "Bearer " + token }
data = { 'message': message }

image = open('img.png', 'rb')
files = { 'imageFile': image }

requests.post("https://notify-api.line.me/api/notify",headers = headers, data = data, files = files)